In [ ]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
from collections import OrderedDict, defaultdict
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import numpy as np
import pandas as pd
#from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
imacscrapedir = '/Users/rayd/workspace/cinema/scrapes/'
airscrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-07-18.csv'
scrape_path = imacscrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   
      
# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# fix up titles, times and days, keep just needed columns, sort rows by movie+day
df.title = df.title.apply(lambda x: '"' + str(x) + '"')
df.times = df.times.str.replace(':00', '')
df.times = df.times.str.replace('|', ',')
df.times = df.times.str.replace('p', ' p.m.')
df.times = df.times.str.replace('a', ' a.m.,')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('date', axis=1)
df['screen'] = df['title'].str.cat(df['format'],sep=", ")
df.screen = df.screen.str.cat(df['mpaa'],sep=", ")
df = df[['theater', 'screen', 'movieday', 'times']]
df.sort_values(["screen", "movieday"], axis =0,
              ascending=True, inplace=True)
df = df.drop_duplicates()

# Ocala market
oc6_df = df.loc[df['theater'].str.contains('Ocala Center 6')]
bv_df = df.loc[df['theater'].str.contains('Belleview')]
mt_df = df.loc[df['theater'].str.contains('Marion')]
omp_df = df.loc[df['theater'].str.contains('Old Mill')]
regaloca_df = df.loc[df['theater'].str.contains('Regal Hollywood')]
rialto_df = df.loc[df['theater'].str.contains('Rialto')]
amc_df = df.loc[df['theater'].str.contains('AMC Lake')]
barn_df = df.loc[df['theater'].str.contains('Barnstorm')]

# Gainesville market
celeb_df = df.loc[df['theater'].str.contains('Celebration')]
hipp_df = df.loc[df['theater'].str.contains('Hippodrome')]
butler_df = df.loc[df['theater'].str.contains('Butler')]
regalgnv_df = df.loc[df['theater'].str.contains('Regal Royal Park')]
uac_df = df.loc[df['theater'].str.contains('Regal UA Cinema 90')]

# Convert each to list of ordered dicts
mt_dict = mt_df.to_dict(into=OrderedDict)
mt_dlist = list(mt_dict.items())
amc_dict = amc_df.to_dict(into=OrderedDict)
amc_dlist = list(amc_dict.items())
barm_dict = barn_df.to_dict(into=OrderedDict)
barn_dlist = list(barm_dict.items())
bv_dict = bv_df.to_dict(into=OrderedDict)
bv_dlist = list(bv_dict.items())
oc6_dict = oc6_df.to_dict(into=OrderedDict)
oc6_dlist = list(oc6_dict.items())
omp_dict = omp_df.to_dict(into=OrderedDict)
omp_dlist = list(omp_dict.items())
regaloca_dict = regaloca_df.to_dict(into=OrderedDict)
regaloca_dlist = list(regaloca_dict.items())
rialto_dict = rialto_df.to_dict(into=OrderedDict)
rialto_dlist = list(rialto_dict.items())
celeb_dict = celeb_df.to_dict(into=OrderedDict)
celeb_dlist = list(celeb_dict.items())
hipp_dict = hipp_df.to_dict(into=OrderedDict)
hipp_dlist = list(hipp_dict.items())
butler_dict = butler_df.to_dict(into=OrderedDict)
butler_dlist = list(butler_dict.items())
regalgnv_dict = regalgnv_df.to_dict(into=OrderedDict)
regalgnv_dlist = list(regalgnv_dict.items())
uac_dict = uac_df.to_dict(into=OrderedDict)
uac_dlist = list(uac_dict.items())

oca_dlists = [mt_dlist, amc_dlist, barn_dlist, bv_dlist, oc6_dlist,
              omp_dlist, regaloca_dlist, rialto_dlist]
gnv_dlists = [hipp_dlist, butler_dlist, celeb_dlist, regalgnv_dlist,
             uac_dlist]

In [ ]:
def fridays_beforepn(dlist):
    
    """Logic to group values from dicts into a narrative for Friday listings"""
    
    if 'AMC Lake Square 12' in list(dlist[0][1].values()):
        print('\nAMC Lake Square 12, 10401-015 US-441, Leesburg\n')
    elif 'Barnstorm Theater' in list(dlist[0][1].values()):
        print('\nBarnstorm Theater, 2720 Brownwood Blvd., The Villages\n')
    elif 'Belleview Cinemas' in list(dlist[0][1].values()):
        print('\nBelleview Cinemas, 10845 SE US Highway 441, Belleview\n')
    elif 'Hippodrome State Theatre' in list(dlist[0][1].values()):
        print('\nHippodrome State Theatre, 25 SE 2nd Place, Gainesville\n')
    elif 'Marion Theatre' in list(dlist[0][1].values()):
        print('\nMarion Theatre, 50 S. Magnolia Ave., Ocala\n')
    elif 'Ocala Center 6' in list(dlist[0][1].values()):
        print('\nOcala Center 6, 2021 Silver Springs Blvd, Ocala\n')
    elif 'Old Mill Playhouse' in list(dlist[0][1].values()):
        print('\nOld Mill Playhouse, 1000 Old Mill Run, The Villages\n')
    elif 'Regal Butler Town Center' in list(dlist[0][1].values()):
        print('\nRegal Butler Town Center, 3101 SW 35th Blvd., Gainesville\n')
    elif 'Regal Celebration Pointe & RPX' in list(dlist[0][1].values()):
        print('\nRegal Celebration Pointe & RPX, 4901 SW 31st Place, Gainesville\n')
    elif 'Regal Hollywood & IMAX - Ocala' in list(dlist[0][1].values()):
        print('\nRegal Hollywood & IMAX-Ocala, 2801 SW 27th Ave., Ocala\n')
    elif 'Regal Royal Park' in list(dlist[0][1].values()):
        print('\nRegal Royal Park, 3702 Newberry Road, Gainesville\n')
    elif 'Regal UA Cinema 90' in list(dlist[0][1].values()):
        print('\nRegal UA Cinema 90, 2830 Hwy. 90 West, Lake City\n')
    elif 'Rialto Theatre Spanish Springs Town Square' in list(dlist[0][1].values()):
        print('\nRialto Theatre Spanish Springs Town Square\n')
                
    #Convert ordered dicts to lists of movies, days and times
    movievals = list(dlist[1][1].values())
    dayvals = list(dlist[2][1].values())
    timevals = list(dlist[3][1].values())
    
    while len(movievals) >= 1:
        if len(movievals) >= 3:
            # same movie all three days; this will be almost all films
            if movievals[0] == movievals[1] and movievals[1] == movievals[2]: # same movie all three days
                # must be Friday-Sunday run
                n = 3 # how many items to remove from lists before loop runs again
                fri = str(timevals[0])
                sat = str(timevals[1])
                sun = str(timevals[2])
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday' and dayvals[2] == 'Sunday':
                    if fri == sat and fri == timevals[2]: # want all common
                        print(movievals[0] + ", today-Sunday at " + fri) 
                    elif fri != sat and fri != sun and sat == sun: # want fri unique
                        print(movievals[0] + ", today at " + fri + ", Saturday-Sunday at " + sat)
                    elif sat != fri and sat != sun and fri == sun: # want sat unique
                        print(movievals[0] + ", today and Sunday at " + fri + ", Saturday at " + sat) 
                    elif sun != fri and sun != sat and fri == sat: # want sun unique
                        print(movievals[0] + ", today and Saturday at " + fri + ", Sunday at " + sun)
                    elif fri != sat and fri != sun and sat !=sun: # want all unique
                        print(movievals[0] + ", today at " + fri + ", Saturday at " + sat + ", Sunday at" + sun)
                    else:
                        print("Show times didn't work out!")
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    print("A problem here")
                    break
            # movie shows just two days 
            elif movievals[0] == movievals[1] and movievals[1] != movievals[2]: 
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    if fri == sat:
                        print(movievals[0] + ", today, Saturday at " + fri)
                    else:
                        print(movievals[0] + ", today at " + fri + ", Saturday at " + sat)
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    if fri == sun:
                        print(movievals[0] + ", today, Sunday at " + fri)
                    else:
                        print(movievals[0] + ", today at " + fri + ", Sunday at " + sun)
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        print(movievals[0] + ", Saturday, Sunday at " + sat)
                    else:
                        print(movievals[0] + ", Saturday at " + sat + ", Sunday at " + sun)
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    print("A problem here")
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                n = 1
                if dayvals[0] == 'Friday':
                    print(movievals[0] + ", today at " + timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday':
                    print(movievals[0] + ", Saturday at " + timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Sunday':
                    print(movievals[0] + ", Sunday at " + timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    print("A problem here")
                    break
            else:
                print("A problem here")
                break
        elif len(movievals) >= 2: # We are down to the last one or two movies; should never happen
            # movie shows just two days 
            if movievals[0] == movievals[1]: 
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    if fri == sat:
                        print(movievals[0] + ", today, Saturday at " + fri)
                    else:
                        print(movievals[0] + ", today at " + fri + ", Saturday at " + sun)
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    if fri == sun:
                        print(movievals[0] + ", today, Sunday at " + timevals[0])
                    else:
                        print(movievals[0] + ", today at " + fri + ", Sunday at " + sun)
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        print(movievals[0] + ", Saturday, Sunday at " + timevals[0])
                    else:
                        print(movievals[0] + ", Saturday at " + sat + ", Sunday at " + sun)
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    print("A problem here")
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                n = 1
                if dayvals[0] == 'Friday':
                    print(movievals[0] + ", today at " + timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday':
                    print(movievals[0] + ", Saturday at " + timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Sunday':
                    print(movievals[0] + ", Sunday at " + timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    print("A problem here")
                    break
        else:        
            n = 1
            if dayvals[0] == 'Friday':
                print(movievals[0] + ", today at " + timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]

            elif dayvals[0] == 'Saturday':
                print(movievals[0] + ", Saturday at " + timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]

            else:
                print(movievals[0] + ", Sunday at " + timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]

In [ ]:
def fridays(dlist):
    
    """Logic to group values from dicts into a narrative for Friday listings"""
    
    global pn # print narrative
    pn = "\n" 
    
    if 'AMC Lake Square 12' in list(dlist[0][1].values()):
        pn += 'AMC Lake Square 12, 10401-015 US-441, Leesburg\n'
    elif 'Barnstorm Theater' in list(dlist[0][1].values()):
        pn += 'Barnstorm Theater, 2720 Brownwood Blvd., The Villages\n'
    elif 'Belleview Cinemas' in list(dlist[0][1].values()):
        pn += 'Belleview Cinemas, 10845 SE US Highway 441, Belleview\n'
    elif 'Hippodrome State Theatre' in list(dlist[0][1].values()):
        pn += 'Hippodrome State Theatre, 25 SE 2nd Place, Gainesville\n'
    elif 'Marion Theatre' in list(dlist[0][1].values()):
        pn += 'Marion Theatre, 50 S. Magnolia Ave., Ocala\n'
    elif 'Ocala Center 6' in list(dlist[0][1].values()):
        pn += 'Ocala Center 6, 2021 Silver Springs Blvd, Ocala\n'
    elif 'Old Mill Playhouse' in list(dlist[0][1].values()):
        pn += 'Old Mill Playhouse, 1000 Old Mill Run, The Villages\n'
    elif 'Regal Butler Town Center' in list(dlist[0][1].values()):
        pn += 'Regal Butler Town Center, 3101 SW 35th Blvd., Gainesville\n'
    elif 'Regal Celebration Pointe & RPX' in list(dlist[0][1].values()):
        pn += 'Regal Celebration Pointe & RPX, 4901 SW 31st Place, Gainesville\n'
    elif 'Regal Hollywood & IMAX - Ocala' in list(dlist[0][1].values()):
        pn += 'Regal Hollywood & IMAX-Ocala, 2801 SW 27th Ave., Ocala\n'
    elif 'Regal Royal Park' in list(dlist[0][1].values()):
        pn += 'Regal Royal Park, 3702 Newberry Road, Gainesville\n'
    elif 'Regal UA Cinema 90' in list(dlist[0][1].values()):
        pn += 'Regal UA Cinema 90, 2830 Hwy. 90 West, Lake City\n'
    elif 'Rialto Theatre Spanish Springs Town Square' in list(dlist[0][1].values()):
        pn += 'Rialto Theatre Spanish Springs Town Square\n'
                
    #Convert ordered dicts to lists of movies, days and times
    movievals = list(dlist[1][1].values())
    dayvals = list(dlist[2][1].values())
    timevals = list(dlist[3][1].values())
    
    while len(movievals) >= 1:
        if len(movievals) >= 3:
            # same movie all three days; this will be almost all films
            if movievals[0] == movievals[1] and movievals[1] == movievals[2]: # same movie all three days
                # must be Friday-Sunday run
                n = 3 # how many items to remove from lists before loop runs again
                fri = str(timevals[0])
                sat = str(timevals[1])
                sun = str(timevals[2])
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday' and dayvals[2] == 'Sunday':
                    if fri == sat and fri == timevals[2]: # want all common
                        pn += movievals[0] + ", today-Sunday at " + fri
                    elif fri != sat and fri != sun and sat == sun: # want fri unique
                        pn += movievals[0] + ", today at " + fri + ", Saturday-Sunday at " + sat
                    elif sat != fri and sat != sun and fri == sun: # want sat unique
                        pn += movievals[0] + ", today and Sunday at " + fri + ", Saturday at " + sat
                    elif sun != fri and sun != sat and fri == sat: # want sun unique
                        pn += movievals[0] + ", today and Saturday at " + fri + ", Sunday at " + sun
                    elif fri != sat and fri != sun and sat !=sun: # want all unique
                        pn += movievals[0] + ", today at " + fri + ", Saturday at " + sat + ", Sunday at" + sun
                    else:
                        pn += "Show times didn't work out!"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # movie shows just two days 
            elif movievals[0] == movievals[1] and movievals[1] != movievals[2]: 
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    if fri == sat:
                        pn += movievals[0] + ", today, Saturday at " + fri
                    else:
                        pn += movievals[0] + ", today at " + fri + ", Saturday at " + sat
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    if fri == sun:
                        pn += movievals[0] + ", today, Sunday at " + fri
                    else:
                        pn += movievals[0] + ", today at " + fri + ", Sunday at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        pn += movievals[0] + ", Saturday, Sunday at " + sat
                    else:
                        pn += movievals[0] + ", Saturday at " + sat + ", Sunday at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                n = 1
                if dayvals[0] == 'Friday':
                    pn += movievals[0] + ", today at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday':
                    pn += movievals[0] + ", Saturday at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Sunday':
                    pn += movievals[0] + ", Sunday at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            else:
                pn += "A problem here"
                break
        elif len(movievals) >= 2: # We are down to the last one or two movies; should never happen
            # movie shows just two days 
            if movievals[0] == movievals[1]: 
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    if fri == sat:
                        pn += movievals[0] + ", today, Saturday at " + fri
                    else:
                        pn += movievals[0] + ", today at " + fri + ", Saturday at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    if fri == sun:
                        pn += movievals[0] + ", today, Sunday at " + timevals[0]
                    else:
                        pn += movievals[0] + ", today at " + fri + ", Sunday at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        pn += movievals[0] + ", Saturday, Sunday at " + timevals[0]
                    else:
                        pn += movievals[0] + ", Saturday at " + sat + ", Sunday at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here")
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                n = 1
                if dayvals[0] == 'Friday':
                    pn += movievals[0] + ", today at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday':
                    pn += movievals[0] + ", Saturday at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Sunday':
                    pn += movievals[0] + ", Sunday at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
        else:        
            n = 1
            if dayvals[0] == 'Friday':
                pn += movievals[0] + ", today at " + str(timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]

            elif dayvals[0] == 'Saturday':
                pn += movievals[0] + ", Saturday at " + str(timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]

            else:
                pn += movievals[0] + ", Sunday at " + str(timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
    return(pn)

In [ ]:
def saturdays(dlist):
    
    """Logic to group values from dicts into a narrative for Saturday listings"""
    
    global pn # print narrative
    pn = "\n"
    
    if 'AMC Lake Square 12' in list(dlist[0][1].values()):
        pn += 'AMC Lake Square 12, 10401-015 US-441, Leesburg\n'
    elif 'Barnstorm Theater' in list(dlist[0][1].values()):
        pn += 'Barnstorm Theater, 2720 Brownwood Blvd., The Villages\n'
    elif 'Belleview Cinemas' in list(dlist[0][1].values()):
        pn += 'Belleview Cinemas, 10845 SE US Highway 441, Belleview\n'
    elif 'Hippodrome State Theatre' in list(dlist[0][1].values()):
        pn += 'Hippodrome State Theatre, 25 SE 2nd Place, Gainesville\n'
    elif 'Marion Theatre' in list(dlist[0][1].values()):
        pn += 'Marion Theatre, 50 S. Magnolia Ave., Ocala\n'
    elif 'Ocala Center 6' in list(dlist[0][1].values()):
        pn += 'Ocala Center 6, 2021 Silver Springs Blvd, Ocala\n'
    elif 'Old Mill Playhouse' in list(dlist[0][1].values()):
        pn += 'Old Mill Playhouse, 1000 Old Mill Run, The Villages\n'
    elif 'Regal Butler Town Center' in list(dlist[0][1].values()):
        pn += 'Regal Butler Town Center, 3101 SW 35th Blvd., Gainesville\n'
    elif 'Regal Celebration Pointe & RPX' in list(dlist[0][1].values()):
        pn += 'Regal Celebration Pointe & RPX, 4901 SW 31st Place, Gainesville\n'
    elif 'Regal Hollywood & IMAX - Ocala' in list(dlist[0][1].values()):
        pn += 'Regal Hollywood & IMAX-Ocala, 2801 SW 27th Ave., Ocala\n'
    elif 'Regal Royal Park' in list(dlist[0][1].values()):
        pn += 'Regal Royal Park, 3702 Newberry Road, Gainesville\n'
    elif 'Regal UA Cinema 90' in list(dlist[0][1].values()):
        pn += 'Regal UA Cinema 90, 2830 Hwy. 90 West, Lake City\n'
    elif 'Rialto Theatre Spanish Springs Town Square' in list(dlist[0][1].values()):
        pn += 'Rialto Theatre Spanish Springs Town Square\n'
                
    #Convert ordered dicts to lists of movies, days and times
    movievals = list(dlist[1][1].values())
    dayvals = list(dlist[2][1].values())
    timevals = list(dlist[3][1].values())
    
    while len(movievals) >= 1:
        if len(movievals) >= 3:
            # same movie all three days; this will be almost all films
            if movievals[0] == movievals[1] and movievals[1] == movievals[2]: # same movie all three days
                # must be Friday-Sunday run
                n = 3 # how many items to remove from lists before loop runs again
                fri = str(timevals[0])
                sat = str(timevals[1])
                sun = str(timevals[2])
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday' and dayvals[2] == 'Sunday':
                    if sat == sun:
                        pn += movievals[0] + ", today, Sunday at " + sat
                    elif sat != sun:
                        pn += movievals[0] + ", today at " + sat + ", Sunday at " + sun
                    else:
                        pn += "Show times didn't work out!"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # movie shows just two days 
            elif movievals[0] == movievals[1] and movievals[1] != movievals[2]: 
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    pn += movievals[0] + ", today at " + sat
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    pn += movievals[0] + ", Sunday at " + fri
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        pn += movievals[0] + ", today, Sunday at " + sat
                    else:
                        pn += movievals[0] + ", today at " + sat + ", Sunday at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here")
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                n = 1
                if dayvals[0] == 'Saturday':
                    pn += movievals[0] + ", today at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                if dayvals[0] == 'Sunday':
                    pn += movievals[0] + ", Sunday at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]                    
                    continue
            else:
                pn += "We have a problem"
                break
        elif len(movievals) >= 2: # We are down to the last one or two movies; should never happen
            # movie shows just two days 
            if movievals[0] == movievals[1]: 
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    pn += movievals[0] + ", today at " + sat
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    pn += movievals[0] + ", Sunday at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        pn += movievals[0] + ", today, Sunday at " + sat
                    else:
                        pn += movievals[0] + ", today at " + sat + ", Sunday at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                n = 1
                if dayvals[0] == 'Saturday':
                    pn += movievals[0] + ", today at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Sunday':
                    pn += movievals[0] + ", Sunday at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here")
                    break
        else:        
            n = 1
            if dayvals[0] == 'Saturday':
                pn += movievals[0] + ", today at " + str(timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]

            else:
                pn += movievals[0] + ", Sunday at " + str(timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
                
    return(pn)

In [1]:
def sundays(dlist):
    
    """Logic to group values from dicts into a narrative for Sunday listings"""
    
    pn global
    pn = "\n"
    
    if 'AMC Lake Square 12' in list(dlist[0][1].values()):
        pn += 'AMC Lake Square 12, 10401-015 US-441, Leesburg\n'
    elif 'Barnstorm Theater' in list(dlist[0][1].values()):
        pn += 'Barnstorm Theater, 2720 Brownwood Blvd., The Villages\n'
    elif 'Belleview Cinemas' in list(dlist[0][1].values()):
        pn += 'Belleview Cinemas, 10845 SE US Highway 441, Belleview\n'
    elif 'Hippodrome State Theatre' in list(dlist[0][1].values()):
        pn += 'Hippodrome State Theatre, 25 SE 2nd Place, Gainesville\n'
    elif 'Marion Theatre' in list(dlist[0][1].values()):
        pn += 'Marion Theatre, 50 S. Magnolia Ave., Ocala\n'
    elif 'Ocala Center 6' in list(dlist[0][1].values()):
        pn += 'Ocala Center 6, 2021 Silver Springs Blvd, Ocala\n'
    elif 'Old Mill Playhouse' in list(dlist[0][1].values()):
        pn += 'Old Mill Playhouse, 1000 Old Mill Run, The Villages\n'
    elif 'Regal Butler Town Center' in list(dlist[0][1].values()):
        pn += 'Regal Butler Town Center, 3101 SW 35th Blvd., Gainesville\n'
    elif 'Regal Celebration Pointe & RPX' in list(dlist[0][1].values()):
        pn += 'Regal Celebration Pointe & RPX, 4901 SW 31st Place, Gainesville\n'
    elif 'Regal Hollywood & IMAX - Ocala' in list(dlist[0][1].values()):
        pn += 'Regal Hollywood & IMAX-Ocala, 2801 SW 27th Ave., Ocala\n'
    elif 'Regal Royal Park' in list(dlist[0][1].values()):
        pn += 'Regal Royal Park, 3702 Newberry Road, Gainesville\n'
    elif 'Regal UA Cinema 90' in list(dlist[0][1].values()):
        pn += 'Regal UA Cinema 90, 2830 Hwy. 90 West, Lake City\n'
    elif 'Rialto Theatre Spanish Springs Town Square' in list(dlist[0][1].values()):
        pn += 'Rialto Theatre Spanish Springs Town Square\n'
                
    #Convert ordered dicts to lists of movies, days and times
    movievals = list(dlist[1][1].values())
    dayvals = list(dlist[2][1].values())
    timevals = list(dlist[3][1].values())
    
    while len(movievals) >= 1:
        if len(movievals) >= 3:
            # same movie all three days; this will be almost all films
            if movievals[0] == movievals[1] and movievals[1] == movievals[2]: # same movie all three days
                # must be Friday-Sunday run
                n = 3 # how many items to remove from lists before loop runs again
                sun = str(timevals[2])
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday' and dayvals[2] == 'Sunday':
                    pn += movievals[0] + ", today at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # movie shows just two days 
            elif movievals[0] == movievals[1] and movievals[1] != movievals[2]: 
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    sun = str(timevals[1])
                    pn += movievals[0] + ", today at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sun = str(timevals[1])
                    pn += movievals[0] + ", today at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
            # it's a one-day show
            else:
                n = 1
                if dayvals[0] == 'Sunday':
                    pn += movievals[0] + ", today at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]                    
                    continue
            
        elif len(movievals) >= 2: # We are down to the last one or two movies; should never happen
            # movie shows just two days 
            if movievals[0] == movievals[1]: 
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    sun = str(timevals[1])
                    pn += movievals[0] + ", today at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sun = str(timevals[1])
                    pn += movievals[0] + ", today at " + sun
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                n = 1
                if dayvals[0] == 'Sunday':
                    pn += movievals[0] + ", Sunday at " + str(timevals[0])
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
        else:        
            n = 1
            if dayvals[0] == 'Sunday':
                pn += movievals[0] + ", today at " + str(timevals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
            else:
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
                continue
    return(pn)


if __name__ == "__main__":
    for dlist in oca_dlists:
        fridays(dlist)
        saturdays(dlist)
        sundays(dlist)
        
    for dlist in gnv_dlists:
        fridays(dlist)
        saturdays(dlist)
        sundays(dlist)

IndentationError: expected an indented block (<ipython-input-1-6da1389a21e0>, line 2)

In [20]:
"""
Once it's working for one threater, put all the above
into a function that can run for each theater, and 
write for files for each day for Ocala and Gainesville.
"""

In [21]:
# Don't really need this now but neat way to join lists
movieday = ['; '.join(map(str, i)) for i in zip(movievals, dayvals)]
moviedaytime = ['; '.join(map(str, i)) for i in zip(movieday, timevals)]

Today-Sunday at 
Today, Saturday at ...
A problem here
